In [1]:
# third-party
import pandas as pd
import numpy as np
import scipy
import pickle

# local
from get_data import transform_overview_on_target, transform_overview_on_overall
from performance import get_fr_detection_performance
from delay_performance import normality_test, plot_inspiration_vs_expiration
from constants import CATEGORICAL_PALETTE


In [2]:
with open('Results/results.pickle', 'rb') as file:
    overview = pickle.load(file)

overview_middle = {}
for id in overview.keys():
    overview_middle[id] = {key: value for key, value in overview[id].items() if key in ['SNBm', 'UALm', 'UARm']}

# remove middle activities
for id in overview.keys():
    del overview[id]["SNBm"]
    del overview[id]["UALm"]
    del overview[id]["UARm"]


### Activity-specific only FR analysis

In [3]:
# Transform overview from participant specific into all participants

overview_all_participants = transform_overview_on_target(overview, target="Activity")
overview_all_participants.keys()

dict_keys(['SNB', 'SGB', 'MIXB', 'STNB', 'MCH', 'SQT', 'AAL', 'AAR', 'ALL', 'ALR', 'UAL', 'UAR', 'SE', 'SS', 'TR'])

In [4]:
performance_all_participants = get_fr_detection_performance(overview_all_participants, target="Activity")
performance_all_participants

,Activity,Sensor,Ratio,Precision,Recall,Mean absolute delay $\pm$ SD,Adjusted delay
0,SNB,MAG,1.00,0.99,1.00,-0.00 $\pm$ 0.27,0 \%
1,SNB,PZT,0.99,0.96,0.95,-0.35 $\pm$ 0.51,-8 \%
2,SGB,MAG,0.99,1.00,0.99,-0.11 $\pm$ 0.55,-1 \%
3,SGB,PZT,1.26,0.77,0.97,-1.11 $\pm$ 1.66,-10 \%
4,MIXB,MAG,1.01,0.99,0.99,-0.05 $\pm$ 0.25,-1 \%
5,MIXB,PZT,0.93,0.97,0.90,-0.46 $\pm$ 0.75,-9 \%
6,STNB,MAG,0.99,1.00,0.99,-0.09 $\pm$ 0.22,-2 \%
7,STNB,PZT,1.02,0.92,0.94,-0.26 $\pm$ 0.78,-6 \%
8,MCH,MAG,0.99,0.97,0.96,0.01 $\pm$ 0.35,0 \%
9,MCH,PZT,1.02,0.87,0.88,-0.10 $\pm$ 0.91,-2 \%


In [5]:
mag = performance_all_participants[performance_all_participants["Sensor"] == "MAG"].loc[:,["Ratio", "Precision", "Recall"]].reset_index(drop=True) 
pzt = performance_all_participants[performance_all_participants["Sensor"] == "PZT"].loc[:,["Ratio", "Precision", "Recall"]].reset_index(drop=True)

a = mag / pzt 
a["Activity"] = performance_all_participants[performance_all_participants["Sensor"] == "MAG"].reset_index(drop=True)["Activity"]
a

,Ratio,Precision,Recall,Activity
0,1.010101,1.031250,1.052632,SNB
1,0.785714,1.298701,1.020619,SGB
2,1.086022,1.020619,1.100000,MIXB
3,0.970588,1.086957,1.053191,STNB
4,0.970588,1.114943,1.090909,MCH
5,1.000000,1.125000,1.126437,SQT
6,1.000000,1.031250,1.021739,AAL
7,0.989583,1.052632,1.032609,AAR
8,1.066667,1.042553,1.119048,ALL
9,1.134146,1.042105,1.194805,ALR


In [6]:
mag["Activity"] = performance_all_participants[performance_all_participants["Sensor"] == "MAG"].reset_index(drop=True)["Activity"]
pzt["Activity"] = performance_all_participants[performance_all_participants["Sensor"] == "PZT"].reset_index(drop=True)["Activity"]
mag

,Ratio,Precision,Recall,Activity
0,1.00,0.99,1.00,SNB
1,0.99,1.00,0.99,SGB
2,1.01,0.99,0.99,MIXB
3,0.99,1.00,0.99,STNB
4,0.99,0.97,0.96,MCH
5,0.98,0.99,0.98,SQT
6,0.95,0.99,0.94,AAL
7,0.95,1.00,0.95,AAR
8,0.96,0.98,0.94,ALL
9,0.93,0.99,0.92,ALR


### Participant-specific only FR analysis

In [7]:
# Transform overview from participant specific into all participants
overview_all_activities = transform_overview_on_target(overview, target="ID")
overview_all_activities.keys()

dict_keys(['7OYX', 'NO15', 'G8B7', 'EPE2', 'HAK8', '1BST', '83J1', 'QMQ7', '9TUL', 'FTD7', 'Y6O3', '2QWT', 'F9AF', 'P4W9', 'W8Z9', 'D4GQ'])

In [8]:
get_fr_detection_performance(overview_all_activities, target="ID")

,ID,Sensor,Ratio,Precision,Recall,Mean absolute delay $\pm$ SD,Adjusted delay
0,7OYX,MAG,1.00,0.99,0.99,-0.09 $\pm$ 0.28,-2 \%
1,7OYX,PZT,1.00,0.80,0.80,-0.38 $\pm$ 1.27,-7 \%
2,NO15,MAG,1.00,1.00,1.00,-0.05 $\pm$ 0.19,-1 \%
3,NO15,PZT,1.13,0.85,0.96,-0.54 $\pm$ 0.62,-12 \%
4,G8B7,MAG,0.83,0.96,0.79,-0.25 $\pm$ 0.49,-7 \%
5,G8B7,PZT,0.96,0.96,0.93,-0.37 $\pm$ 0.39,-10 \%
6,EPE2,MAG,0.96,0.99,0.95,0.25 $\pm$ 0.39,5 \%
7,EPE2,PZT,1.01,0.89,0.90,-0.46 $\pm$ 1.00,-10 \%
8,HAK8,MAG,0.99,0.93,0.92,-0.11 $\pm$ 0.35,-3 \%
9,HAK8,PZT,0.94,0.80,0.76,0.31 $\pm$ 0.88,8 \%


### Overall FR Analysis

In [9]:
# Transform overview from participant specific into all participants
overview_all = transform_overview_on_overall(overview)
overview_all.keys()

dict_keys(['MAG', 'Airflow', 'PZT'])

In [10]:
get_fr_detection_performance(overview_all, target=None)

,Sensor,Ratio,Precision,Recall,Mean absolute delay $\pm$ SD,Adjusted delay
0,MAG,0.97,0.97,0.95,-0.03 $\pm$ 0.35,NaN
1,PZT,0.98,0.90,0.88,-0.18 $\pm$ 0.86,NaN


In [11]:
fr_detection_all = get_fr_detection_performance(overview, target="both")
fr_detection_all

,ID,Activity,Sensor,Ratio,Precision,Recall
0,7OYX,SNB,MAG,1.00,1.00,1.00
1,7OYX,SNB,PZT,1.00,0.96,0.96
2,7OYX,SGB,MAG,1.00,1.00,1.00
3,7OYX,SGB,PZT,1.00,0.92,0.92
4,7OYX,MIXB,MAG,1.00,1.00,1.00
...,...,...,...,...,...,...
473,D4GQ,UAR,PZT,0.95,1.00,0.95
474,D4GQ,SE,MAG,1.00,1.00,1.00
475,D4GQ,SE,PZT,1.00,1.00,1.00
476,D4GQ,SS,MAG,0.89,1.00,0.89


In [12]:
fr_mag = fr_detection_all[fr_detection_all["Sensor"]=="MAG"]
fr_pzt = fr_detection_all[fr_detection_all["Sensor"]=="PZT"]

In [13]:
metric = "Ratio"
normality_test(fr_mag[metric], sensor=metric, type="All data")
normality_test(fr_pzt[metric], sensor=metric, type="All data")

In [14]:
metric = "Precision"
normality_test(fr_mag[metric], sensor=metric, type="All data")
normality_test(fr_pzt[metric], sensor=metric, type="All data")

In [15]:
metric = "Recall"
normality_test(fr_mag[metric], sensor=metric, type="All data")
normality_test(fr_pzt[metric], sensor=metric, type="All data")

In [16]:
from stats import wilcoxon

wilcoxon(fr_mag["Ratio"], fr_pzt["Ratio"], "Ratio")
wilcoxon(fr_mag["Precision"], fr_pzt["Precision"], "Precision")
wilcoxon(fr_mag["Recall"], fr_pzt["Recall"], "Recall")


Ratio: W(239)=8036.5, p=0.51
Precision: W(239)=1155.5, p<0.001
Recall: W(239)=2945.0, p<0.001


In [17]:
from stats import wilcoxon

wilc = pd.DataFrame(columns=["Activity", "Metric", "Statistic", "p-value", "Report"])

for metric in ["Ratio", "Precision", "Recall"]:
    for activity in mag["Activity"].unique():
        new_entry = {}
        new_entry["Activity"] = activity
        new_entry["Metric"] = metric
        new_entry["Statistic"], new_entry["p-value"] = scipy.stats.wilcoxon(fr_mag[fr_mag["Activity"]==activity][metric].values, fr_pzt[fr_pzt["Activity"]==activity][metric].values)
        new_entry["Report"] =  wilcoxon(fr_mag[fr_mag["Activity"]==activity][metric].values, fr_pzt[fr_pzt["Activity"]==activity][metric].values, metric, return_string=True)
        wilc.loc[len(wilc)] = new_entry
        
wilc

/Users/anasofiacc/dev/M-RESP/.venv/lib/python3.9/site-packages/scipy/stats/_morestats.py:4088: UserWarning:

Exact p-value calculation does not work if there are zeros. Switching to normal approximation.

/Users/anasofiacc/dev/M-RESP/.venv/lib/python3.9/site-packages/scipy/stats/_morestats.py:4102: UserWarning:

Sample size too small for normal approximation.

/Users/anasofiacc/dev/M-RESP/.venv/lib/python3.9/site-packages/scipy/stats/_morestats.py:4088: UserWarning:

Exact p-value calculation does not work if there are zeros. Switching to normal approximation.

/Users/anasofiacc/dev/M-RESP/.venv/lib/python3.9/site-packages/scipy/stats/_morestats.py:4088: UserWarning:

Exact p-value calculation does not work if there are zeros. Switching to normal approximation.

/Users/anasofiacc/dev/M-RESP/.venv/lib/python3.9/site-packages/scipy/stats/_morestats.py:4102: UserWarning:

Sample size too small for normal approximation.

/Users/anasofiacc/dev/M-RESP/.venv/lib/python3.9/site-packages/scipy/

,Activity,Metric,Statistic,p-value,Report
0,SNB,Ratio,13.0,0.481683,"Ratio: W(16)=13.0, p=0.48"
1,SGB,Ratio,4.0,0.003614,"Ratio: W(16)=4.0, p=0.00"
2,MIXB,Ratio,4.0,0.027848,"Ratio: W(15)=4.0, p=0.03"
3,STNB,Ratio,17.5,0.941627,"Ratio: W(16)=17.5, p=0.94"
4,MCH,Ratio,38.0,0.362686,"Ratio: W(16)=38.0, p=0.36"
5,SQT,Ratio,44.0,0.593618,"Ratio: W(16)=44.0, p=0.59"
6,AAL,Ratio,47.5,0.753257,"Ratio: W(16)=47.5, p=0.75"
7,AAR,Ratio,21.5,0.540291,"Ratio: W(16)=21.5, p=0.54"
8,ALL,Ratio,41.0,0.470338,"Ratio: W(16)=41.0, p=0.47"
9,ALR,Ratio,40.0,0.255989,"Ratio: W(16)=40.0, p=0.26"


In [18]:
#wilc[wilc["Metric"]=="Ratio"][wilc["p-value"] < 0.05].sort_values(by="Statistic")
wilc[wilc["Activity"]=="SGB"]

,Activity,Metric,Statistic,p-value,Report
1,SGB,Ratio,4.0,0.003614,"Ratio: W(16)=4.0, p=0.00"
16,SGB,Precision,0.0,0.002089,"Precision: W(16)=0.0, p=0.00"
31,SGB,Recall,0.0,0.058782,"Recall: W(16)=0.0, p=0.06"


In [19]:
9.58e-3

0.00958

### Overall delay analysis

In [20]:
delays_i_all_mag, delays_e_all_mag = [], []
delays_i_all_pzt, delays_e_all_pzt = [], []

for activity in overview_all_participants.keys():

    delays_i_all_mag += overview_all_participants[activity]["MAG"]["delay_i"] 
    delays_e_all_mag += overview_all_participants[activity]["MAG"]["delay_e"]

    delays_i_all_pzt += overview_all_participants[activity]["PZT"]["delay_i"]
    delays_e_all_pzt += overview_all_participants[activity]["PZT"]["delay_e"]

In [21]:
print(f"min delay mag {min(delays_i_all_mag + delays_e_all_mag, key=abs)}")
print(f"min delay pzt {min(delays_i_all_pzt + delays_e_all_pzt, key=abs)}")

print(f"max delay mag {max(delays_i_all_mag + delays_e_all_mag, key=abs)}")
print(f"max delay pzt {max(delays_i_all_pzt + delays_e_all_pzt, key=abs)}")

print(f"mean delay mag {round(np.mean(delays_i_all_mag + delays_e_all_mag), 2)} +/- {np.round(np.std(delays_i_all_mag + delays_e_all_mag), 2)}")
print(f"mean delay pzt {np.round(np.mean(delays_i_all_pzt + delays_e_all_pzt), 2)} +/- {np.round(np.std(delays_i_all_pzt + delays_e_all_pzt), 2)}")

min delay mag -0.0
min delay pzt -0.0
max delay mag -2.88
max delay pzt 5.05
mean delay mag -0.03 +/- 0.35
mean delay pzt -0.18 +/- 0.86


In [22]:
normality_test(delays_i_all_mag + delays_e_all_mag, sensor="MAG", type="inspiration+expiration")
normality_test(delays_i_all_pzt + delays_e_all_pzt, sensor="PZT", type="inspiration+expiration")

### Event-specific delay analysis

In [23]:
normality_test(delays_i_all_mag, sensor="MAG", type="inspiration")
normality_test(delays_e_all_mag, sensor="MAG", type="expiration")
normality_test(delays_i_all_pzt, sensor="PZT", type="inspiration", categorical_palette=CATEGORICAL_PALETTE[2:])
normality_test(delays_e_all_pzt, sensor="PZT", type="expiration", categorical_palette=CATEGORICAL_PALETTE[2:])

In [24]:
delays_df = pd.DataFrame(columns=["Sensor", "Type", "Delay"])
delays_df["Delay"] = delays_i_all_mag + delays_e_all_mag + delays_i_all_pzt + delays_e_all_pzt
delays_df["Sensor"] = ["MAG"] * len(delays_i_all_mag + delays_e_all_mag) + ["PZT"] * len(delays_i_all_pzt + delays_e_all_pzt)
delays_df["Type"] = ["Inhalation"] * len(delays_i_all_mag) + ["Exhalation"] * len(delays_e_all_mag) + ["Inhalation"] * len(delays_i_all_pzt) + ["Exhalation"] * len(delays_e_all_pzt)
delays_df


,Sensor,Type,Delay
0,MAG,Inhalation,-0.10
1,MAG,Inhalation,-0.05
2,MAG,Inhalation,-0.19
3,MAG,Inhalation,-0.07
4,MAG,Inhalation,-0.16
...,...,...,...
12494,PZT,Exhalation,-0.62
12495,PZT,Exhalation,-1.34
12496,PZT,Exhalation,-0.87
12497,PZT,Exhalation,-0.69


In [25]:
plot_inspiration_vs_expiration(delays_df)

#### Statistical testing: comparing inhalation vs expiration

In [26]:
from stats import mannwhitney
print(len(delays_i_all_mag) + len(delays_e_all_mag))
print(len(delays_i_all_pzt) + len(delays_e_all_pzt))

mannwhitney(delays_i_all_mag, delays_e_all_mag, "MAG")
mannwhitney(delays_i_all_pzt, delays_e_all_pzt, "PZT")

6541
5958
MAG: U(1635)=5422437.5, p=0.33
PZT: U(1490)=4422868.0, p=0.83


### Correlation between strap-band perimeter and FR performance

In [27]:
strap_thorax_ratio_dict = {
    '7OYX': 0.94,
    'NO15': 0.99,
    'G8B7': 0.96,
    'EPE2': 0.93,
    'HAK8': 0.94,
    '1BST': 0.98,
    '83J1': 0.97,
    'QMQ7': 0.96,
    '9TUL': 0.9,
    'FTD7': 0.9,
    'Y6O3': 0.92,
    '2QWT': 0.85,
    'F9AF': 0.87,
    'P4W9': 0.96,
    'W8Z9': 0.94,
    'D4GQ': 0.92
}

In [28]:
strap_thorax_performance_df = get_fr_detection_performance(overview, target="both")
strap_thorax_performance_df = strap_thorax_performance_df[strap_thorax_performance_df["Sensor"] == "MAG"]
strap_thorax_performance_df['strap-to-thorax ratio'] = strap_thorax_performance_df["ID"].apply(lambda id: strap_thorax_ratio_dict[id])
strap_thorax_performance_df


,ID,Activity,Sensor,Ratio,Precision,Recall,strap-to-thorax ratio
0,7OYX,SNB,MAG,1.00,1.00,1.00,0.94
2,7OYX,SGB,MAG,1.00,1.00,1.00,0.94
4,7OYX,MIXB,MAG,1.00,1.00,1.00,0.94
6,7OYX,STNB,MAG,0.97,1.00,0.97,0.94
8,7OYX,MCH,MAG,1.00,1.00,1.00,0.94
...,...,...,...,...,...,...,...
468,D4GQ,ALR,MAG,0.67,1.00,0.67,0.92
470,D4GQ,UAL,MAG,1.05,0.96,1.00,0.92
472,D4GQ,UAR,MAG,0.95,1.00,0.95,0.92
474,D4GQ,SE,MAG,1.00,1.00,1.00,0.92


In [29]:
from stats import spearmanr
spearmanr(strap_thorax_performance_df["Ratio"].values, strap_thorax_performance_df["strap-to-thorax ratio"].values, "strap-to-thorax ratio")
#scipy.stats.spearmanr(strap_thorax_performance_df["Ratio"].values, strap_thorax_performance_df["strap-to-thorax ratio"].values)

strap-to-thorax ratio: $\rho$(239)=-0.0, p=0.83


### Analysis on alternative positioning

In [30]:
overview_middle_all_participants = transform_overview_on_target(overview_middle, target="Activity")
performance_middle_all_participants = get_fr_detection_performance(overview_middle_all_participants, target="Activity")
performance_middle_all_participants = performance_middle_all_participants[performance_middle_all_participants["Sensor"]=="MAG"]
performance_middle_all_participants

,Activity,Sensor,Ratio,Precision,Recall,Mean absolute delay $\pm$ SD,Adjusted delay
0,SNBm,MAG,0.99,1.00,0.98,0.06 $\pm$ 0.20,1 \%
2,UALm,MAG,0.95,0.98,0.92,0.30 $\pm$ 0.51,7 \%
4,UARm,MAG,1.01,0.97,0.98,0.33 $\pm$ 0.51,8 \%


In [31]:
performance_middle_vs_left = pd.DataFrame(columns=performance_middle_all_participants.columns)

for activity in performance_middle_all_participants["Activity"].unique():
    new_entry = performance_all_participants[(performance_all_participants["Activity"]==activity[:-1]) & (performance_all_participants["Sensor"]=="MAG")]
    performance_middle_vs_left = pd.concat([performance_middle_vs_left, new_entry], ignore_index=True)

    new_entry = performance_middle_all_participants[performance_middle_all_participants["Activity"]==activity]
    performance_middle_vs_left = pd.concat([performance_middle_vs_left, new_entry], ignore_index=True)

performance_middle_vs_left.drop(columns=["Sensor", "Mean absolute delay $\pm$ SD", "Adjusted delay"], inplace=True)
performance_middle_vs_left

,Activity,Ratio,Precision,Recall
0,SNB,1.00,0.99,1.00
1,SNBm,0.99,1.00,0.98
2,UAL,1.00,0.97,0.97
3,UALm,0.95,0.98,0.92
4,UAR,0.98,0.98,0.97
5,UARm,1.01,0.97,0.98


In [32]:
delays_middle_i_all_mag, delays_middle_e_all_mag = [], []
delays_middle_i_all_pzt, delays_middle_e_all_pzt = [], []

for activity in overview_middle_all_participants.keys():

    delays_middle_i_all_mag += overview_middle_all_participants[activity]["MAG"]["delay_i"] 
    delays_middle_e_all_mag += overview_middle_all_participants[activity]["MAG"]["delay_e"]

    delays_middle_i_all_pzt += overview_middle_all_participants[activity]["PZT"]["delay_i"]
    delays_middle_e_all_pzt += overview_middle_all_participants[activity]["PZT"]["delay_e"]

print(f"min delay mag {min(delays_middle_i_all_mag + delays_middle_e_all_mag, key=abs)}")
print(f"min delay pzt {min(delays_middle_i_all_pzt + delays_middle_e_all_pzt, key=abs)}")

print(f"max delay mag {max(delays_middle_i_all_mag + delays_middle_e_all_mag, key=abs)}")
print(f"max delay pzt {max(delays_middle_i_all_pzt + delays_middle_e_all_pzt, key=abs)}")

print(f"mean delay mag {round(np.mean(delays_middle_i_all_mag + delays_middle_e_all_mag), 2)} +/- {np.round(np.std(delays_middle_i_all_mag + delays_middle_e_all_mag), 2)}")
print(f"mean delay pzt {np.round(np.mean(delays_middle_i_all_pzt + delays_middle_e_all_pzt), 2)} +/- {np.round(np.std(delays_middle_i_all_pzt + delays_middle_e_all_pzt), 2)}")

min delay mag -0.0
min delay pzt -0.0
max delay mag 2.37
max delay pzt 4.43
mean delay mag 0.18 +/- 0.4
mean delay pzt -0.49 +/- 0.9
